# Overall Explanation of Generating Simulated Sequence File 

explanation

In [3]:
import sys
print(sys.executable)

/home/jp394/miniforge3/envs/SMaTH/bin/python


In [4]:
from graphviz import Digraph

ModuleNotFoundError: No module named 'graphviz'

In [74]:
from graphviz import Digraph
from IPython.display import Image

# Create a Digraph object
dot = Digraph(comment='Simple Flow Chart')

# Add nodes and edges to the graph
dot.node('A', 'Start')
dot.node('B', 'Step 1')
dot.node('C', 'Step 2')
dot.node('D', 'End')

dot.edges(['AB', 'BC', 'CD'])

# Render the graph to a PNG file
dot.render('flow_chart', format='png', cleanup=True)

# Display the PNG file in the notebook
Image(filename='flow_chart.png')

ModuleNotFoundError: No module named 'graphviz'

# User Defined Functions

In [23]:
import vcf
import subprocess
import pysam
import pyranges 

# Load VCF Files

def vcf_to_bed(vcf_path, bed_path):
    # Open VCF file
    vcf_reader = vcf.Reader(open(vcf_path, 'r'))

    # Open BED file for writing
    with open(bed_path, 'w') as bed_file:
        # Iterate through VCF records and write to BED file
        for record in vcf_reader:
            chrom = record.CHROM
            start = record.POS - 1  # Convert to 0-based coordinates for BED format
            end = record.POS
            info = record.INFO

            bed_file.write(f"{chrom}\t{start}\t{end}\t{info}\n")


In [30]:
# Original source code is from https://www.biostars.org/p/13516/

from collections import namedtuple, defaultdict
from pybedtools import BedTool
import argparse

Point    = namedtuple('Point', ['id', 'pos', 'type'])
Interval = namedtuple('Interval', ['chrom', 'start', 'end'])


def report_interval(chrom, start, end, num_files, files_with_interval):
    print("\t".join([chrom, str(start), str(end), str(len(files_with_interval.keys()))]), end="")
    for i in range(0, num_files):
        if i in files_with_interval:
            print("\t1", end="")
        else:
            print("\t0", end="")


def merge(file):
    """
    Merge features in a BED/GFF/VCF into non-overlapping intervals
    """
    start = -1
    end   = -1
    chrom = None
    for feature in BedTool(file):
        if feature.start - end > 0 or end < 0 or feature.chrom != chrom:
            if start >= 0:
                yield Interval(chrom, start, end)
            start = feature.start
            end   = feature.end
            chrom = feature.chrom
        elif feature.end > end:
            end = feature.end
    yield Interval(chrom, start, end)


def load_and_sort_points(files):
    """
    """
    file_id = 0
    chrom_points = defaultdict(list)
    # for each input file, first merge the features into non-overlapping
    # intervals using merge().  Each non-overlapping feature is then 
    # broken up into discrete "Points": one for the start and one for the end.
    for file in files:
        # merge the file and split features into points
        for feature in merge(file):
            s = Point(file_id, feature.start, "start")
            e = Point(file_id, feature.end,   "end")
            chrom_points[feature.chrom].append(s)
            chrom_points[feature.chrom].append(e)
        file_id += 1
    
    # sort the points in for each chrom 
    for chrom in chrom_points:
        chrom_points[chrom].sort(key=lambda i: i.pos)
    return chrom_points


def load_genome(genome):
    chrom_sizes = {}
    for line in open(genome, 'r'):
        fields = line.strip().split("\t")
        if len(fields) > 1:
            chrom_sizes[fields[0]] = fields[1]

    return chrom_sizes


def nway(files, genome):
    """
    Assumptions: input files must contain non-overlapping intervals
    
    1. Example using already-merged files:
    $ cat a.merged 
    chr1	6	20
    chr1	22	30
    
    $ cat b.merged 
    chr1	12	32
    
    $ cat c.merged 
    chr1	8	15
    chr1	32	34
    
    
    $ ./nway-cluster.py a.merged b.merged c.merged
    #chr	st	ed	num	a	b	c 
    chr1	0	6 	0 	0 	0 	0
    chr1	6	8 	1 	1 	0 	0
    chr1	8	12 	2 	1 	0 	1
    chr1	12	15 	3 	1 	1 	1
    chr1	15	20 	2 	1 	1 	0
    chr1	20	22 	1 	0 	1 	0
    chr1	22	30 	2 	1 	1 	0
    chr1	30	32 	1 	0 	1 	0
    chr1	32	34 	1 	0 	0 	1
    
    
    2. Example using un-merged, yet sorted files:
    $ cat a.bed 
    chr1	6	12
    chr1	10	20
    chr1	22	27
    chr1	24	30
    
    $ cat b.bed 
    chr1	12	32
    chr1	14	30
    
    $ cat c.bed 
    chr1	8	15
    chr1	10	14
    chr1	32	34
    
    $ ./nway-cluster.py a.bed b.bed c.bed
    #chr	st	ed	num	a	b	c 
    chr1	0	6 	0 	0 	0 	0
    chr1	6	8 	1 	1 	0 	0
    chr1	8	12 	2 	1 	0 	1
    chr1	12	15 	3 	1 	1 	1
    chr1	15	20 	2 	1 	1 	0
    chr1	20	22 	1 	0 	1 	0
    chr1	22	30 	2 	1 	1 	0
    chr1	30	32 	1 	0 	1 	0
    chr1	32	34 	1 	0 	0 	1
    
    
    3. Thanks to pybedtools, it works with BAM files as well.
       But I hope you have a machine with lots of RAM.
    ./nway-cluster.py 1.bam 2.bam 3.bam
    
    """
    num_files = len(files)
    
    # 1. load each point from each interval in each file into
    #    a hash keyed by chrom.  
    # 2. sort the points in asecnding order for each chrom
    chrom_points = load_and_sort_points(files)
    if genome is not None:
        chrom_sizes  = load_genome(genome)
    
    # 3. Rip through the points and find shared intervals
    for chrom in chrom_points:
        files_with_interval = {}
        prev_point = 0
        for point in chrom_points[chrom]:
            # report the current interval if we've moved at all along the chrom.
            if point.pos > prev_point:
                report_interval(chrom, prev_point, point.pos, num_files, files_with_interval)
            # if we're at a start, we add the current file to the active list of files. 
            # otherwise, an end point means we can drop the current file.
            if point.type == "start":
                files_with_interval[point.id] = 1
            else:
                del files_with_interval[point.id]
            prev_point = point.pos

        # if requested, handle the interval from the last observed point to the end of the chrom
        if genome is not None and point.pos < chrom_sizes[chrom]:
            report_interval(chrom, point.pos, chrom_sizes[chrom], num_files, files_with_interval)


# 1. Bed File Manipulation

## 1.1. Generate Bed File from VCF File

In [26]:
vcf_to_bed("results/xTea/shortread/mosaic/GIAB/Illumina300x/HG002/Alu/HG002.GRCh38.300x_ALU.vcf", "results/xTea/shortread/mosaic/GIAB/Illumina300x/HG002/Alu/HG002.GRCh38.300x_ALU.bed")
vcf_to_bed("results/xTea/shortread/mosaic/GIAB/Illumina300x/HG003/Alu/HG003.GRCh38.300x_ALU.vcf", "results/xTea/shortread/mosaic/GIAB/Illumina300x/HG003/Alu/HG003.GRCh38.300x_ALU.bed")

## 1.2. Load Bed Files

In [ ]:
pyranges.read_bed("results/xTea/shortread/mosaic/GIAB/Illumina300x/HG003/Alu/HG003.GRCh38.300x_ALU.bed")

## 1.3. Calculate Intersect between two bed files

## 1.4. Calculate AF from the bam file based on bed file

# Remove Spikes from BAM File

In [6]:

def remove_spikes(input_bam, bed_file, output_bam):
    try:
        # Open input BAM file
        with pysam.AlignmentFile(input_bam, "rb") as input_bam_file:
            # Open BED file for regions to keep
            with open(bed_file, "r") as bed:
                # Create a list of regions from the BED file
                regions = [line.strip().split('\t') for line in bed]

                # Fetch reads overlapping the specified regions
                filtered_reads = input_bam_file.fetch(regions=regions)

                # Open output BAM file for writing
                with pysam.AlignmentFile(output_bam, "wb", header=input_bam_file.header) as output_bam_file:
                    # Write filtered reads to output BAM file
                    for read in filtered_reads:
                        output_bam_file.write(read)

        print(f"Spikes removed successfully. Output saved to {output_bam}")

    except Exception as e:
        print(f"Error: {e}")

def extract_regions(input_bam, bed_file, output_bam):
    # Step 2: Extract regions from the first BAM file
    with pysam.AlignmentFile(input_bam, "rb") as bam_file:
        regions = pysam.TabixFile(bed_file)
        extracted_reads = [read for read in bam_file.fetch() if regions.fetch(read.reference_name, read.reference_start, read.reference_end)]

    # Write the extracted reads to a new BAM file
    with pysam.AlignmentFile(output_bam, "wb", header=bam_file.header) as output_bam_file:
        for read in extracted_reads:
            output_bam_file.write(read)


def simulate_insertions(reference_genome, extracted_bam, output_prefix):
    try:
        # Open reference genome
        with pysam.FastaFile(reference_genome) as ref_fasta:
            # Open extracted BAM file
            with pysam.AlignmentFile(extracted_bam, "rb") as input_bam_file:
                # Open output BAM file for writing simulated reads
                with pysam.AlignmentFile(output_prefix + ".bam", "wb", header=input_bam_file.header) as output_bam_file:
                    # Iterate over reads in the input BAM file
                    for read in input_bam_file:
                        # Simulate insertions (modify read sequence, qualities, etc. as needed)
                        simulated_read = simulate_insertion(read, ref_fasta)

                        # Write the simulated read to the output BAM file
                        output_bam_file.write(simulated_read)

        print(f"Insertions simulated successfully. Output saved to {output_prefix}.bam")

    except Exception as e:
        print(f"Error: {e}")

def simulate_insertion(read, ref_fasta):
    # Modify the read to simulate an insertion (customize based on your needs)
    # For example, you can extend the read sequence, qualities, etc.
    simulated_read = read.copy()
    simulated_read.query_sequence += "INSERTED_SEQUENCE"
    simulated_read.query_qualities += [30] * len("INSERTED_SEQUENCE")

    # Adjust alignment information if necessary
    simulated_read.reference_end += len("INSERTED_SEQUENCE")

    return simulated_read


def merge_bams(original_bam, simulated_bam, merged_bam):
    # Step 4: Merge BAM files
    with pysam.AlignmentFile(original_bam, "rb") as original_file, \
         pysam.AlignmentFile(simulated_bam + '.sam', "rb") as simulated_file, \
         pysam.AlignmentFile(merged_bam, "wb", header=original_file.header) as output_bam:
        for read in original_file:
            output_bam.write(read)
        for read in simulated_file:
            output_bam.write(read)

    # Step 5: Sort and index merged BAM file
    pysam.sort('-o', merged_bam.replace('.bam', '_sorted.bam'), merged_bam)
    pysam.index(merged_bam.replace('.bam', '_sorted.bam'))

# if __name__ == "__main__":
#     # Specify file paths
#     input_bam = "/path/to/your/original.bam"
#     bed_file = "/path/to/your/simulated_insertions.bed"
#     output_bam = "/path/to/your/extracted_reads.bam"
#     reference_genome = "/path/to/your/reference_genome.fasta"
#     output_prefix = "/path/to/your/simulated_insertions"
#     merged_bam = "/path/to/your/final_merged.bam"

#     # Extract regions from the original BAM file
#     extract_regions(input_bam, bed_file, output_bam)

#     # Simulate insertions
#     simulate_insertions(reference_genome, output_bam, output_prefix)

#     # Merge the original and simulated BAM files
#     merge_bams(input_bam, output_prefix + '.bam', merged_bam)